# Socket connection with serialization

### Server

In [1]:
using Sockets, Serialization, AppliSales, AppliAR, AppliGeneralLedger, DataFrames

In [2]:
@async begin
    server = listen(IPv4(0), 2014)
    while true
        sock = accept(server)
        @async while isopen(sock)
            data = deserialize(sock)
            #println(typeof(data))
            client = connect(ip"192.168.2.40", 2015)
            if data isa Vector{AppliSales.Order} && isopen(client)
                entries = AppliAR.process(data)
                serialize(client, entries)
            elseif data isa Vector{BankStatement} && isopen(client)
                unpaid_inv = AppliAR.retrieve_unpaid_invoices()
                entries = AppliAR.process(unpaid_inv, data)
                serialize(client, entries)
            end
            #client = connect(ip"10.152.183.82", 2001).
        end
    end
end

#wait(Condition())

Task (runnable) @0x00007f26582f8be0

### Aging report

In [3]:
r1 = AppliAR.report()
result = DataFrame(r1)

,id_inv,csm,inv_date,amount,days
,String,String,Dates…,Float64,Dates…
1,A-1005,Scrooge Investment Bank,"Date(""2021-08-26"")",1210.0,Day(0)
2,A-1006,Duck City Chronicals,"Date(""2021-08-26"")",2420.0,Day(0)
3,A-1007,Donalds Hardware Store,"Date(""2021-08-26"")",1210.0,Day(0)
